<a href="https://colab.research.google.com/github/xiaoyi001yeye/JXLS-extension/blob/master/LangChain_Docs_Chinese_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/xiaoyi001yeye/LangChain-Docs-Chinese.git

Cloning into 'LangChain-Docs-Chinese'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 46 (delta 12), reused 24 (delta 4), pack-reused 0
Unpacking objects: 100% (46/46), 250.69 KiB | 3.69 MiB/s, done.


In [ ]:
%%bash
pip install polib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import datetime
import hashlib
import base64
import hmac
import json

class get_result(object):
    def __init__(self,host):
        
        
        
        # 以下为POST请求
        self.Host = host
        self.RequestUri = "/v2/ots"
        # 设置url
        # print(host)
        self.url="http://"+host+self.RequestUri
        self.HttpMethod = "POST"
        self.Algorithm = "hmac-sha256"
        self.HttpProto = "HTTP/1.1"

        # 设置当前时间
        curTime_utc = datetime.datetime.utcnow()
        self.Date = self.httpdate(curTime_utc)
        # 设置业务参数
        # 语种列表参数值请参照接口文档：https://www.xfyun.cn/doc/nlp/niutrans/API.html
        self.Text="How old are you"
        self.BusinessArgs={
                "from": "en",
                "to": "zh",
            }

    def hashlib_256(self, res):
        m = hashlib.sha256(bytes(res.encode(encoding='utf-8'))).digest()
        result = "SHA-256=" + base64.b64encode(m).decode(encoding='utf-8')
        return result

    def httpdate(self, dt):
        """
        Return a string representation of a date according to RFC 1123
        (HTTP/1.1).

        The supplied date must be in UTC.

        """
        weekday = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"][dt.weekday()]
        month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep",
                 "Oct", "Nov", "Dec"][dt.month - 1]
        return "%s, %02d %s %04d %02d:%02d:%02d GMT" % (weekday, dt.day, month,
                                                        dt.year, dt.hour, dt.minute, dt.second)

    def generateSignature(self, digest):
        signatureStr = "host: " + self.Host + "\n"
        signatureStr += "date: " + self.Date + "\n"
        signatureStr += self.HttpMethod + " " + self.RequestUri \
                        + " " + self.HttpProto + "\n"
        signatureStr += "digest: " + digest
        signature = hmac.new(bytes(self.Secret.encode(encoding='utf-8')),
                             bytes(signatureStr.encode(encoding='utf-8')),
                             digestmod=hashlib.sha256).digest()
        result = base64.b64encode(signature)
        return result.decode(encoding='utf-8')

    def init_header(self, data):
        digest = self.hashlib_256(data)
        #print(digest)
        sign = self.generateSignature(digest)
        authHeader = 'api_key="%s", algorithm="%s", ' \
                     'headers="host date request-line digest", ' \
                     'signature="%s"' \
                     % (self.APIKey, self.Algorithm, sign)
        #print(authHeader)
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Method": "POST",
            "Host": self.Host,
            "Date": self.Date,
            "Digest": digest,
            "Authorization": authHeader
        }
        return headers

    def get_body(self,text):
        
        content = str(base64.b64encode(self.Text.encode('utf-8') if text is None else text.encode('utf-8') ), 'utf-8')
        postdata = {
            "common": {"app_id": self.APPID},
            "business": self.BusinessArgs,
            "data": {
                "text": content,
            }
        }
        body = json.dumps(postdata)
        #print(body)
        return body

    def call_url(self):
        if self.APPID == '' or self.APIKey == '' or self.Secret == '':
            print('Appid 或APIKey 或APISecret 为空！请打开demo代码，填写相关信息。')
        else:
            code = 0
            body=self.get_body()
            headers=self.init_header(body)
            #print(self.url)
            response = requests.post(self.url, data=body, headers=headers,timeout=8)
            status_code = response.status_code
            #print(response.content)
            if status_code!=200:
                # 鉴权失败
                print("Http请求失败，状态码：" + str(status_code) + "，错误信息：" + response.text)
                print("请根据错误信息检查代码，接口文档：https://www.xfyun.cn/doc/nlp/niutrans/API.html")
            else:
                # 鉴权成功
                respData = json.loads(response.text)
                print(respData)
                # 以下仅用于调试
                code = str(respData["code"])
                if code!='0':
                    print("请前往https://www.xfyun.cn/document/error-code?code=" + code + "查询解决办法")
    def tanslate(self,text):
        body=self.get_body(text)
        headers=self.init_header(body)
        #print(self.url)
        response = requests.post(self.url, data=body, headers=headers,timeout=8)
        status_code = response.status_code
        #print(response.content)
        if status_code!=200:
            # 鉴权失败
            print("Http请求失败，状态码：" + str(status_code) + "，错误信息：" + response.text)
            print("请根据错误信息检查代码，接口文档：https://www.xfyun.cn/doc/nlp/niutrans/API.html")
        else:
            # 鉴权成功
            data = json.loads(response.text)
            return data.get("data").get("result").get("trans_result").get("dst")

# if __name__ == '__main__':
#     ##示例:  host="ntrans.xfyun.cn"域名形式
#     host = "ntrans.xfyun.cn"
#     #初始化类
#     gClass=get_result(host)
#     gClass.call_url()

def tanslate(text):
  host = "ntrans.xfyun.cn"
  gClass=get_result(host)
  return gClass.tanslate(text=text)

In [ ]:
print(tanslate("Deployments"))

部署


In [ ]:
import csv
import polib
from typing import Tuple

# 定义翻译函数
def translate_text(text: str) -> str:
    # 在这里实现翻译功能，此处只作为示例
    return tanslate(str)
print(tanslate("Deployments"))
#定义 PO 文件路径和 CSV 文件路径
po_file_path = "/content/LangChain-Docs-Chinese/docs_gettext/messages.PO"
csv_file_path = "/content/message.csv"

# 使用 polib 包解析 PO 文件
po = polib.pofile(po_file_path)
messages = [(entry.msgid, entry.msgstr) for entry in po]

# 翻译每个消息并保存到 CSV 文件
with open(csv_file_path, "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    # 写入 CSV 文件头部
    writer.writerow(["msgid", "msgstr"])
    for msgid, msgstr in messages:
        # 翻译消息
        try:
            msgstr_translated = tanslate(msgid)
        except:
            msgstr_translated = ""
        # 写入 CSV 文件
        writer.writerow([msgid, msgstr_translated])

print("翻译完成，已保存到 CSV 文件中。")


部署
翻译完成，已保存到 CSV 文件中。
